In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [4]:
sub_input, val_input,sub_target,val_target = train_test_split(train_input, train_target, test_size=0.2,random_state=42)

In [5]:
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.00600624, 0.0059824 , 0.004987  , 0.00598431, 0.00498605]), 'score_time': array([0.00097609, 0.        , 0.0009973 , 0.00099683, 0.00099778]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [17]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [26]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params)

In [27]:
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
  

In [28]:
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [30]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [31]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [32]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [33]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), 'max_depth': range(5,20,1), 'min_samples_split': range(2,100,10)}

In [34]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params)
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': range(5, 20),
                         

In [35]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [36]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [37]:
from scipy.stats import uniform, randint

In [39]:
rgen=randint(0,10)
rgen.rvs(10)

array([6, 1, 9, 2, 1, 5, 2, 3, 7, 2])

In [40]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110,  92, 108, 113, 100, 105, 106,  88,  90,  88], dtype=int64))

In [41]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.21529345, 0.11939474, 0.7897567 , 0.97214588, 0.3161057 ,
       0.43287679, 0.11334672, 0.6322259 , 0.12710957, 0.88650493])

In [44]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [45]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [46]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [47]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [48]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
